In [42]:
import numpy as np
import math
from collections import namedtuple

from onms_decoder import OnmsDecoder
from awgn import Awgn
from bpsk import Bpsk
from awgn_llr_adapter import AwgnLlrAdapter


In [46]:
Settings = namedtuple('Settings', 'max_iter rejections_count snr_array')

In [66]:
def get_sigma(snr):
    return np.sqrt(10 ** (-snr/10)/2)

def simulate(settings, codeword, modulation, channel, decoder, decoder_adapter=None):
    
    simulations_count = len(settings.snr_array)
    fers = []
    is_max_iteration_reached = []
    
    for snr in settings.snr_array:
        
        sigma = get_sigma(snr)
        channel.sigma = sigma
        decoder_adapter.sigma = sigma
        
        errors_count = 0
        iterations_count = 0
        
        while (errors_count <= settings.rejections_count) and (iterations_count < settings.max_iter): 
            modulated = modulation.modulate(codeword);
            transmitted = channel.simulate(modulated);
            
            if decoder_adapter is not None:
                transmitted = decoder_adapter.adapt(transmitted)

            decoded = decoder.decode(transmitted)
            errors_count += not decoded[0]
            iterations_count += 1
        
        fer = errors_count / iterations_count
        fers.append(fer)
        is_max_iteration_reached.append(settings.max_iter == iterations_count)
        
    return {'snr': settings.snr_array.copy, 'fers':fers, 'is_max_iteration_reached':is_max_iteration_reached }

In [67]:
H = np.array([[1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 0]])
codeword = np.array([0, 0, 1, 1, 1, 0])

decoder = OnmsDecoder(H);
channel = Awgn()
modulation = Bpsk()
adapter = AwgnLlrAdapter()

settings = Settings(100000, 30, [0, 0.5, 1, 1.5, 10])
result = simulate(settings, codeword, modulation, channel, decoder, adapter)
print(result['fers'])

[0.07848101265822785, 0.07598039215686274, 0.043909348441926344, 0.038130381303813035, 0.0]
